In [21]:
import pandas as pd
from itertools import groupby
from datetime import datetime

In [8]:
with open("../results.txt") as f:
    content = f.readlines()

In [9]:
content = [line.strip() for line in content]

In [96]:
chunks = (list(g) for k, g in groupby(content, key=lambda x: x != '--blame-end--') if k)

In [97]:
def parse_blame(chunk):
    author = chunk[1][7:]
    author_mail = chunk[2][13:-1]
    author_time_int = chunk[3][12:]
    author_time = datetime.fromtimestamp(int(author_time_int))
    filename = chunk[-2][9:]
    comment_text = chunk[-1]
    comment = comment_text[comment_text.find("TODO"):]
    return author, author_mail, author_time, filename, comment

In [98]:
data = []

for chunk in chunks:
    data.append(list(parse_blame(chunk)))

In [99]:
df = pd.DataFrame(data, columns=['author', 'author_mail', 'author_time', 'filename', 'comment'])

In [93]:
df.to_csv("pytorch.csv", index=False)

In [101]:
def apply_tags(comment):
    comment = comment.lower()
    tags = []
    
    uncertainty_words = {"uncertainty": ["?", "maybe", "perhaps", "should we", "probably", "might", "not sure"]}
    hacky_words = {"hacky": ["temporary", "hack", "hacky"]}
    fixme_words = {"fix": ["fixme", "fix", "bug", "incorrect"]}
    dependency_words = {"dependency": ["once", "when", "blocked"]}
    
    for tw in [uncertainty_words, hacky_words, fixme_words, dependency_words]:
        for k, v in tw.items():
            for w in v:
                if w in comment:
                    tags.append(k)
    return tags

In [103]:
df['tags'] = df.comment.map(apply_tags)

In [105]:
df

,author,author_mail,author_time,filename,comment,tags
0,Rong Rong,rongr@fb.com,2020-09-08 17:08:00,.circleci/cimodel/data/binary_build_definition...,TODO cuda images should consolidate into tag-b...,[]
1,Edward Yang,ezyang@fb.com,2019-06-04 07:21:19,.circleci/cimodel/data/pytorch_build_definitio...,TODO expand this to cover all the USE_* that w...,[]
2,Karl Ostmo,kostmo@gmail.com,2019-02-22 20:10:22,.circleci/cimodel/pytorch_build_definitions.py,TODO: Eliminate the special casing for docker ...,[]
3,Karl Ostmo,kostmo@gmail.com,2019-02-22 20:10:22,.circleci/cimodel/pytorch_build_definitions.py,"TODO When merging the caffe2 and pytorch jobs,...","[uncertainty, dependency]"
4,Karl Ostmo,kostmo@gmail.com,2019-02-22 11:22:14,.circleci/cimodel/pytorch_build_definitions.py,TODO This is a hack to special case some confi...,[hacky]
...,...,...,...,...,...,...
1416,Tzu-Wei Huang,huang.dexter@gmail.com,2019-05-08 14:03:04,torch/utils/tensorboard/_pytorch_graph.py,TODO: compute correct memory usage and CPU tim...,[dependency]
1417,Lara Haidar,haidar.lara@gmail.com,2020-03-29 23:12:32,torch/utils/tensorboard/_pytorch_graph.py,TODO: move outside of torch.onnx?,[uncertainty]
1418,Tzu-Wei Huang,huang.dexter@gmail.com,2019-04-25 21:22:34,torch/utils/tensorboard/_pytorch_graph.py,TODO: See if we can extract GPU vs CPU informa...,[]
1419,Tzu-Wei Huang,huang.dexter@gmail.com,2019-08-26 10:38:30,torch/utils/tensorboard/summary.py,TODO: expose other parameters in the future.,[]
